# Taller 5: Implementar en Notebook Jupyter una red de colas de un Centro de Diagnóstico Automotriz que emiten la revisión técnico-mecánica

Diego Fernando Palacios Hoyos

Codigo: 16003727

In [4]:
import math as m
from datetime import datetime
import time
import numpy as np

def generarU():
    global Xa
    a=1103515245
    c=1
    m=2**31-1
    Xo=Xa
    Xn=(a*Xo+c)%m
    ui=Xn/m
    Xa=Xn
    return ui

def generarU2():
    global Xa2
    a=1103515245
    c=1
    m=2**31-1
    Xo=Xa
    Xn=(a*Xo+c)%m
    ui=Xn/m
    Xa=Xn
    return ui


def exponencial():
    U=generarU()
    return (-1/lamda)*m.log(1-U)


c=[1.0,1.0]

def ck(k):
    cr=0.0
    for i in range(0,k):
        cr+=(c[i]*c[k-1-i])/((i+1.0)*(2.0*i+1.0))
    c.append(cr)
    
for i in range(2,1001):
    ck(i)  


for i in range(2,1001):
    ck(i)  

def erf(z):
    r=0.0
    for k in range(0,1000):
        r+=(c[k]/(2*k+1))*((m.sqrt(m.pi)/2.0)*z)**(2.0*k+1.0)
    return r

def normal(med,var):
    un=generarU()
    resul=med + var*m.sqrt(2.0)*erf(2.0*un-1.0)
    if resul<0.0:
        resul=0.0
    if resul>20:
        resul=20
    return resul

def menor():
    m=0
    for i in range(1,len(p)):
        if Tsuc[p[i]]<Tsuc[p[m]]:
            m=i
    return p[m]

def llegadaCliente(tsuc):
    global n_med_n1,n1,NLL1,LL1,t,Tsuc
    n_med_n1=n_med_n1+n1*(tsuc-t)
    n1=n1+1
    NLL1=NLL1+1
    LL1.append(0)
    LL1[NLL1]=tsuc
    t=tsuc
    Y=exponencial()    
    if t+Y<T:
        Tsuc['tLL1']=t+Y
    if n1==1:
        Y=normal(M1,V1)
        Tsuc['ts1']=t+Y
        
def servicioNodo1(tsuc):
    global n_med_n1,n1,Ns1,S1,n_med_n2,n2,NLL2,LL2,Tsuc,n_med_n3,n3,NLL3,LL3,t
    n_med_n1=n_med_n1+n1*(tsuc-t)
    n1=n1-1
    Ns1=Ns1+1
    S1[Ns1]=tsuc
    U=generarU()
    if U<0.4:
        n_med_n2=n_med_n2+n2*(tsuc-t)
        n2=n2+1
        NLL2=NLL2+1
        LL2[NLL2]=tsuc
        if n2==1:
            Z=exponencial()
            Tsuc['ts2']=tsuc+Z
    else:
        n_med_n3=n_med_n3+n3*(tsuc-t)
        n3=n3+1
        NLL3=NLL3+1
        LL3[NLL3]=tsuc
        if n3==1:
            W=normal(MED31,VAR31)
            Tsuc['ts3']=tsuc+W
    t=tsuc
    if n1>0:
        S=normal(M1,V1)
        Tsuc['ts1']=t+S
    
def servicioNodo2(tsuc):
    global n_med_n1,n1,Ns1,S1,n_med_n2,n2,NLL2,LL2,Tsuc,n_med_n3,n3,NLL3,LL3,t,Ns2,S2,t
    n_med_n2=n_med_n2+n2*(tsuc-t)
    
    n2=n2-1
    Ns2=Ns2+1
    S2.append(0)
    S2[Ns2]=tsuc
    if n2>0:
        Y=exponencial()
        Tsuc['ts2']=tsuc+Y
        n_med_n3=n_med_n3+n3*(tsuc-t)
        n3=n3+1
        NLL3=NLL3+1
        LL3[NLL3]=tsuc
    if n3==1:
        W=normal(MED31,VAR31)
        Tsuc['ts3']=tsuc+W
        t=tsuc
        
def servicioNodo3(tsuc):
    global n_med_n3,n3,t,Ns3,t
    n_med_n3=n_med_n3+n3*(tsuc-t)
    
    n3=n3-1
    Ns3=Ns3+1
    S3.append(0)
    S3[Ns3]=tsuc
    if n3>0:
        if n3<5:
            R=normal(MED31,VAR31)
        else:
            R=normal(MED32,VAR32)
        Tsuc['ts3']=tsuc+R
    t=tsuc
    

In [5]:
a=datetime.now()
Xa=a.microsecond
lamda=0.7

a2=datetime.now()
Xa2=a2.microsecond

M1=0.2
V1=0.05
MED31=2
VAR31=1
MED32=2
VAR32=1
m2=2
    
M=2**15
Tsuc={'tLL1':float(M),'ts1':float(M),'ts2':float(M),'ts3':float(M)}

p=list(Tsuc.keys())
t=0
Tp=0
tsuc=0
NLL1,NLL2,NLL3,Ns1,Ns2,Ns3,n1,n2,n3=0,0,0,0,0,0,0,0,0
i=1000
LL1,LL2,LL3=[0 for x in range(i)],[0 for x in range(i)],[0 for x in range(i)]
S1,S2,S3=[0 for x in range(i)],[0 for x in range(i)],[0 for x in range(i)]
n_med_n1,n_med_n2,n_med_n3=0,0,0
T=500

X=float(exponencial())

if X>T:
    Tp,t_medio_sistema=0,0
    n_med_n1,n_med_n2,n_med_n3=0,0,0
else:
    llegadaCliente(X)
    while((Tsuc['tLL1'] or Tsuc['ts1'] or Tsuc['ts2'] or Tsuc['ts3'])!=M):
        if n3<0:
            print("")
        
        minn=Tsuc[menor()]
       
        if minn==Tsuc['tLL1']:
            tsuc=Tsuc['tLL1']
            Tsuc['tLL1']=M
            llegadaCliente(tsuc)
        if minn==Tsuc['ts1']:
            tsuc=Tsuc['ts1']
            Tsuc['ts1']=M
            servicioNodo1(tsuc)
        if minn==Tsuc['ts2']:
            tsuc=Tsuc['ts2']
            Tsuc['ts2']=M
            servicioNodo2(tsuc)
        if minn==Tsuc['ts3']:
            tsuc=Tsuc['ts3']
            Tsuc['ts3']=M
            servicioNodo3(tsuc)

Tp=max(0,t-T)
acumulo1,acumulo2,acumulo3=0,0,0
ind=0
while ind<NLL1:
    acumulo1=acumulo1+S1[ind]-LL1[ind]
    ind=ind+1
ind=0
while ind<NLL2:
    acumulo2=acumulo2+S2[ind]-LL2[ind]
    ind=ind+1
ind=0
while ind<NLL3:
    acumulo3=acumulo3+S2[ind]-LL3[ind]
    ind=ind+1


n_med_n1=n_med_n1/t
n_med_n2=n_med_n2/t
n_med_n3=n_med_n3/t



print("MODELO DE RED DE COLAS\n")
print("Centro de Diagnostico Automotriz\n")
print ("Numero medio de clientes en servidor 1= %f"%n_med_n1)
print ("Numero medio de clientes en servidor 2= %f"%n_med_n2)
print ("Numero medio de clientes en servidor 3= %f"%n_med_n3)





MODELO DE RED DE COLAS

Centro de Diagnostico Automotriz

Numero medio de clientes en servidor 1= 0.150652
Numero medio de clientes en servidor 2= 0.289708
Numero medio de clientes en servidor 3= 6.022932
